In [4]:
cd C:\Users\allis\Desktop\Data Science\Project Showcase\Alogrithms\Big_Heist\data

C:\Users\allis\Desktop\Data Science\Project Showcase\Alogrithms\Big_Heist\data


In [5]:
import math


def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def check_solution(travel_list, df, speed=30.):
    """
    Check a bank robber algorithm solution
    
    input:
    ------
    travel_list: a list of integers
        Your solution to the bank robber problem
        Bank IDs are row indeces into the df passed
    df: pd.DataFrame
        The algorithm input formatted dataframe 
        (id,x_coordinate,y_coordinate,money,time (hr))
    speed: float
        The km/h speed of travel
    """
    t_remaining = 24.
    score = 0.
    assert len(travel_list) == len(set(travel_list)), (
        "Your travel list must have unique IDs!"
    )
    prev = travel_list[0]
    for e in travel_list:
        row = df.iloc[e]
        score += row['money']
        t_remaining -= row['time (hr)']
        dist = distance(
            row['x_coordinate'], row['y_coordinate'],
            df.iloc[prev]['x_coordinate'],
            df.iloc[prev]['y_coordinate'],
        )
        t_remaining -= dist / speed
        prev = e
    assert t_remaining >= 0, (
        f"Used more than 24h! Time left: {t_remaining}"
    )
    # still gotta get to (0, 0)
    dist = distance(row['x_coordinate'],row['y_coordinate'],0,0)
    final_t = t_remaining - (dist / speed)
    assert final_t >= 0, (
        f"Not enough time to get to helicopter!\n"
        f"Time left after last bank: {t_remaining}\n"
        f"Distance to (0,0) helipad: {dist}\n"
    )
    print(f"Time Remaining: {final_t}")
    return score

Checklist

- Steal as much money as possible within 24h. 

- Must return to helicopter (0,0)

- Function must output bank IDs

- Our top speed is 30km/h

- Start at the most $$$ bank


In [6]:
import pandas as pd 
import numpy as np 
import seaborn as sns
df = pd.read_csv('bank_data.csv')#.head(500) #Dont forget to make it normal size after
df.info()
#sns.scatterplot(data=df , x ='x_coordinate',y= 'y_coordinate', alpha = 0.4 )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            10000 non-null  int64  
 1   x_coordinate  10000 non-null  float64
 2   y_coordinate  10000 non-null  float64
 3   money         10000 non-null  int64  
 4   time (hr)     10000 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 390.8 KB


In [7]:
def distancebetween(X1,Y1,X2=0,Y2=0):
    return np.sqrt((X2-X1)**2 + (Y2-Y1)**2)

heistlist = []
time_left = 24
#setting up the first bank
rich_bankid = df.money.idxmax()                                 #finds most expensive bank 
heistlist.append(rich_bankid)                                   #adds most expesive bank to list
X = float(df['x_coordinate'].loc[df.index == rich_bankid])
Y = float(df['y_coordinate'].loc[df.index == rich_bankid])
first_time = float(df['time (hr)'].loc[df.index == rich_bankid]) #robbery time for first bank
time_left -= first_time                                          #removes time to rob first bank 
main_heli_dist = distancebetween(X,Y)                            #distance from helicopter

temp_df = df.copy()
temp_df.drop(int(rich_bankid),axis=0,inplace = True) #drops first bank

temp_df['distbetween'] = distancebetween(temp_df['x_coordinate'],temp_df['y_coordinate'],X,Y) #finds distance between first point 
temp_df['travel_time'] = temp_df['distbetween']/30 #calculates travel_time
temp_df['money/h'] = temp_df['money']/(temp_df['time (hr)']+temp_df['travel_time']) #money per hour value of each bank from current bank
temp_df['total_time'] = temp_df['time (hr)']+temp_df['travel_time']
temp_df['heli_time'] = (distancebetween(temp_df['x_coordinate'],temp_df['y_coordinate']))/30

while time_left > 0:
    if temp_df.empty:  #If df empty then stop 
        remaining_time = time_left - escape_time
        print(f'remaining_time :{remaining_time}')
        print(heistlist)
        break

    next_bankid = temp_df['money/h'].idxmax()             #finds next bank
    time_taken = float(temp_df['total_time'].loc[temp_df.index == next_bankid])
    
    if time_left - (time_taken + float(temp_df['heli_time'].loc[temp_df.index == next_bankid]))< 0: #checks if there is enough time to escape
        temp_df.drop(int(next_bankid),axis=0,inplace = True) 
        

    else:
        heistlist.append(next_bankid)          #adds next bank
        time_left -= time_taken                #takes time off
        X = float(temp_df['x_coordinate'].loc[temp_df.index == next_bankid])        #change new start variable
        Y = float(temp_df['y_coordinate'].loc[temp_df.index == next_bankid])
        escape_time = float(temp_df['heli_time'].loc[temp_df.index == next_bankid]) #Counts total time taken for if statement ^

        temp_df.drop(int(next_bankid),axis=0,inplace = True)                        #drops next bank

        #Update temp_df with new starting point
        temp_df['distbetween'] = distancebetween(temp_df['x_coordinate'],temp_df['y_coordinate'],X,Y) #finds distance between the next point  
        temp_df['travel_time'] = temp_df['distbetween']/30
        temp_df['money/h'] = temp_df['money']/(temp_df['time (hr)']+temp_df['travel_time'])
        temp_df['total_time'] = temp_df['time (hr)']+temp_df['travel_time']

remaining_time :0.00021606857764822074
[4848, 5933, 7560, 487, 7343, 3193, 6468, 4762, 299, 2331, 4499, 8525, 9653, 4287, 6528, 8690, 4757, 4789, 4696, 7064, 8436, 6097, 8287, 3798, 3516, 433, 9546, 3613, 7544, 9241, 2928, 9378, 5296, 8469, 2346, 2741, 9049, 8966, 9401, 3297, 8206, 2656, 9736, 5155, 4627, 4492, 6535, 2190, 8562, 3914, 5135, 951, 1372, 58, 5610, 9195, 4987, 5562, 4906, 7087, 6281, 7074, 7258, 6740, 2729, 7701, 7764, 3026, 1757, 8355, 7689, 7877, 4605, 6104, 613, 5719, 8375, 3605, 1455, 6254, 557, 7595, 9881, 670, 9275, 9469, 2590, 5381, 5126, 6156, 1397, 8909, 7625, 7769, 2, 7801, 5627, 4610, 6317, 5622, 517, 9290, 7265, 8908, 1447, 5356, 7531, 1997, 6759, 781, 6216, 3186, 4234, 2028, 8125, 9779, 2243, 1976, 3466, 865, 444, 7583, 3005, 9583, 524, 9228, 4807, 3089, 7665, 5166, 8231, 8846, 9640, 2626, 6022, 5298, 507, 7772, 2472, 70, 1684, 790, 4494, 3585, 488, 2226, 9880, 5563, 5971, 7494, 2639, 6375, 754, 359, 1966, 3971, 3340, 6478, 2458, 6987, 9120, 4345, 5184, 757, 4

In [8]:
check_solution(heistlist, df, speed=30.)

Time Remaining: 0.00021606857763817322


13746500.0